In [8]:
import matplotlib.pyplot as plt
import pandas as pd
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv

In [4]:
conda install -c conda-forge folium

Solving environment: ...working... done

## Package Plan ##

  environment location: D:\Anaconda\envs\geo-env

  added / updated specs:
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.4.1               |             py_0          26 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         184 KB  conda-forge
Note: you may need to restart the kernel to use updated packages.
    certifi-2020.6.20          |   py38h32f6830_0         151 KB  conda-forge
    folium-0.11.0              |             py_0          61 KB  conda-forge
    openssl-1.1.1g             |       he774522_0         5.7 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         6.1 MB

The following NEW packages will be INSTALLED:

  branca             conda-forge/noarch::branca-0.4.1-py_0
  folium    



==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda




The following packages will be SUPERSEDED by a higher-priority channel:

  openssl                                         pkgs/main --> conda-forge




ca-certificates-2020 | 184 KB    |            |   0% 
ca-certificates-2020 | 184 KB    | 8          |   9% 
ca-certificates-2020 | 184 KB    | ########## | 100% 

openssl-1.1.1g       | 5.7 MB    |            |   0% 
openssl-1.1.1g       | 5.7 MB    |            |   0% 
openssl-1.1.1g       | 5.7 MB    | #8         |  19% 
openssl-1.1.1g       | 5.7 MB    | #####7     |  58% 
openssl-1.1.1g       | 5.7 MB    | ########## | 100% 

folium-0.11.0        | 61 KB     |            |   0% 
folium-0.11.0        | 61 KB     | #####2     |  52% 
folium-0.11.0        | 61 KB     | ########## | 100% 

branca-0.4.1         | 26 KB     |            |   0% 
branca-0.4.1         | 26 KB     | ########## | 100% 

certifi-2020.6.20    | 151 KB    |            |   0% 
certifi-2020.6.20    | 151 KB    | ########## | 100% 
Preparing transaction: ...working

In [6]:
import folium

In [7]:
#getting data from the url
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
html_content = page.text

In [9]:
#load data into bs4
soup = BeautifulSoup(html_content,'lxml')
table = soup.find("table",{"class":"wikitable sortable"})

In [10]:
#use pandas to read the table
df = pd.read_html(str(table))

In [11]:
df = df[0]

In [12]:
#drop the row which Borough is not assigned
df = df[df.Borough != 'Not assigned']

In [13]:
df.sort_values('Postal Code')

,Postal Code,Borough,Neighborhood
9,M1B,Scarborough,"Malvern, Rouge"
18,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
27,M1E,Scarborough,"Guildwood, Morningside, West Hill"
36,M1G,Scarborough,Woburn
45,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
107,M9P,Etobicoke,Westmount
116,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
143,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [14]:
#reset the column name
df.columns = ['PostalCode','Borough','Neighborhood']

In [15]:
#read csv file and import to data pandaframe
geocode = 'https://cocl.us/Geospatial_data'
geodata = pd.read_csv(geocode)

In [16]:
geodata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
geocode_data = pd.merge(df,geodata,left_on='PostalCode',right_on='Postal Code').drop('Postal Code',axis = 1)
geocode_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [18]:
df.groupby('Borough').count()['Neighborhood']

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
East York            5
Etobicoke           12
Mississauga          1
North York          24
Scarborough         17
West Toronto         6
York                 5
Name: Neighborhood, dtype: int64

In [20]:
#Select only Toronto Neighbourhoods
Toronto_data = geocode_data[geocode_data['Borough'].str.contains('Toronto')]

Toronto_data = Toronto_data.reset_index(drop=True)
Toronto_data.groupby('Borough').count()['Neighborhood']

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
West Toronto         6
Name: Neighborhood, dtype: int64

In [21]:
CLIENT_ID = 'ZYLC4Q3I000O4R32DVJWJJTOTHCGC4O02TXYEPLDAS211SPQ' # your Foursquare ID
CLIENT_SECRET = 'OPKHF1MTRWKRHVR2DAV0IT1IK2H2XZDXJYTCNHVY5L44T55H' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
   
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
toronto_venues = getNearbyVenues(names=Toronto_data['Neighborhood'],
                                latitudes=Toronto_data['Latitude'],
                                longitudes=Toronto_data['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [25]:
toronto_venues.shape

(1622, 7)

In [26]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",17,17,17,17,17,17
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,65,65,65,65,65,65
Christie,16,16,16,16,16,16
Church and Wellesley,77,77,77,77,77,77
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,35,35,35,35,35,35


In [27]:
toronto_venues['Venue Category'].unique()[:100]

array(['Bakery', 'Coffee Shop', 'Distribution Center', 'Spa', 'Pub',
       'Park', 'Restaurant', 'Breakfast Spot', 'Gym / Fitness Center',
       'Historic Site', 'Farmers Market', 'Performing Arts Venue',
       'Chocolate Shop', 'Dessert Shop', 'French Restaurant', 'Café',
       'Yoga Studio', 'Theater', 'Event Space', 'Shoe Store',
       'Ice Cream Shop', 'Art Gallery', 'Cosmetics Shop',
       'Asian Restaurant', 'Electronics Store', 'Bank', 'Beer Store',
       'Wine Shop', 'Antique Shop', 'Italian Restaurant',
       'Sushi Restaurant', 'Creperie', 'Beer Bar', 'Arts & Crafts Store',
       'Burrito Place', 'Mexican Restaurant', 'Hobby Shop', 'Diner',
       'Fried Chicken Joint', 'Discount Store', 'Smoothie Shop',
       'Sandwich Place', 'Gym', 'Bar', 'College Auditorium',
       'College Cafeteria', 'Comic Shop', 'Clothing Store', 'Tea Room',
       'Plaza', 'Music Venue', 'Pizza Place', 'Thai Restaurant',
       'Ramen Restaurant', 'College Rec Center', 'Sporting Goods Shop

In [28]:
"Chinese Restaurant" in toronto_venues['Venue Category'].unique()


True

In [30]:
onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhoods'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

print(onehot.shape)
onehot.head()

(1622, 234)


,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
groupData = onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(groupData.shape)
groupData

(39, 234)


,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.015385,0.000000,0.000000,0.015385,0.000000,0.000000,0.015385
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.012987,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012987,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.012987,0.000000,0.025974
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,...,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.028571,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [33]:
ChineseRestaurant = groupData[["Neighborhoods","Chinese Restaurant"]]


In [34]:
ChineseRestaurant.head()

,Neighborhoods,Chinese Restaurant
0,Berczy Park,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.0
2,"Business reply mail Processing Centre, South C...",0.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0
4,Central Bay Street,0.0


In [38]:
ChineseRestaurant.sort_values('Chinese Restaurant',ascending = False)

,Neighborhoods,Chinese Restaurant
21,"North Toronto West, Lawrence Park",0.052632
30,"St. James Town, Cabbagetown",0.045455
38,"University of Toronto, Harbord",0.028571
37,"Toronto Dominion Centre, Design Exchange",0.010000
14,"Harbourfront East, Union Station, Toronto Islands",0.010000
13,"Garden District, Ryerson",0.010000
27,Roselawn,0.000000
23,"Queen's Park, Ontario Provincial Government",0.000000
24,"Regent Park, Harbourfront",0.000000
25,"Richmond, Adelaide, King",0.000000


In [43]:
pip install -U scikit-learn

Note: you may need to restart the kernel to use updated packages.Collecting threadpoolctl>=2.0.0



In [49]:
len(groupData[groupData["Chinese Restaurant"] > 0])

6

In [78]:
from sklearn.cluster import KMeans
toclusters = 3

to_clustering = ChineseRestaurant.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=toclusters, random_state=1)
kmeans.fit_transform(to_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:36]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])

In [82]:
mergedData = ChineseRestaurant.copy()

# add clustering labels
mergedData["Cluster Labels"] = kmeans.labels_

In [83]:
mergedData = mergedData.join(toronto_venues.set_index("Neighborhood"), on="Neighborhoods")

print(mergedData.shape)
mergedData.head()

(1622, 9)


,Neighborhoods,Chinese Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.0,0,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
0,Berczy Park,0.0,0,43.644771,-79.373306,The Keg Steakhouse + Bar - Esplanade,43.646712,-79.374768,Restaurant
0,Berczy Park,0.0,0,43.644771,-79.373306,Meridian Hall,43.646292,-79.376022,Concert Hall
0,Berczy Park,0.0,0,43.644771,-79.373306,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant
0,Berczy Park,0.0,0,43.644771,-79.373306,Hockey Hall Of Fame (Hockey Hall of Fame),43.646974,-79.377323,Museum


In [93]:
mergedData

,Neighborhoods,Chinese Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.000000,0,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
25,"Richmond, Adelaide, King",0.000000,0,43.650571,-79.384568,Civello Salon & Spa,43.650020,-79.389400,Salon / Barbershop
25,"Richmond, Adelaide, King",0.000000,0,43.650571,-79.384568,Copacabana Grilled Brazilian,43.648333,-79.388151,Brazilian Restaurant
25,"Richmond, Adelaide, King",0.000000,0,43.650571,-79.384568,Canoe,43.647452,-79.381320,Restaurant
25,"Richmond, Adelaide, King",0.000000,0,43.650571,-79.384568,Fune Japanese Restaurant,43.648514,-79.386457,Japanese Restaurant
...,...,...,...,...,...,...,...,...,...
38,"University of Toronto, Harbord",0.028571,2,43.662696,-79.400049,Harbord Bakery & Calandria,43.662519,-79.404443,Bakery
38,"University of Toronto, Harbord",0.028571,2,43.662696,-79.400049,FLOCK Rotisserie + Greens,43.662637,-79.403798,Comfort Food Restaurant
38,"University of Toronto, Harbord",0.028571,2,43.662696,-79.400049,Innis Cafe,43.665401,-79.399715,Café
38,"University of Toronto, Harbord",0.028571,2,43.662696,-79.400049,Coach House Printing,43.666320,-79.400277,Bookstore


In [85]:
mergedData.sort_values(["Cluster Labels"], inplace=True)
mergedData.head()

,Neighborhoods,Chinese Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.0,0,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
25,"Richmond, Adelaide, King",0.0,0,43.650571,-79.384568,Civello Salon & Spa,43.650020,-79.389400,Salon / Barbershop
25,"Richmond, Adelaide, King",0.0,0,43.650571,-79.384568,Copacabana Grilled Brazilian,43.648333,-79.388151,Brazilian Restaurant
25,"Richmond, Adelaide, King",0.0,0,43.650571,-79.384568,Canoe,43.647452,-79.381320,Restaurant
25,"Richmond, Adelaide, King",0.0,0,43.650571,-79.384568,Fune Japanese Restaurant,43.648514,-79.386457,Japanese Restaurant


In [86]:
lat_toronto = Toronto_data['Latitude'].mean()
lon_toronto = Toronto_data['Longitude'].mean()
print('The geographical coordinates of Toronto are {}, {}'.format(lat_toronto, lon_toronto))

The geographical coordinates of Toronto are 43.66713498717949, -79.38987324871795


In [94]:
map_clusters = folium.Map(location=[lat_toronto, lon_toronto],zoom_start=14)

# set color scheme for the clusters


# add markers to the map
markers_colors={}
markers_colors[2] = 'red'
markers_colors[1] = 'blue'
markers_colors[0] = 'green'

for lat, lon, cluster in zip(mergedData['Neighborhood Latitude'], mergedData['Neighborhood Longitude'], mergedData['Cluster Labels']):
    
    
    folium.CircleMarker(
        [lat, lon],
        radius=5,
       
        color =markers_colors[cluster],
        fill_color=markers_colors[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [88]:
#Cluster 1 area

mergedData.loc[(mergedData['Cluster Labels'] ==1) & (mergedData['Venue Category'] == 'Chinese Restaurant') ]

,Neighborhoods,Chinese Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
30,"St. James Town, Cabbagetown",0.045455,1,43.667967,-79.367675,Tender Trap Restaurant,43.667724,-79.369485,Chinese Restaurant
21,"North Toronto West, Lawrence Park",0.052632,1,43.715383,-79.405678,C'est Bon,43.716785,-79.400406,Chinese Restaurant
30,"St. James Town, Cabbagetown",0.045455,1,43.667967,-79.367675,China Gourmet,43.664180,-79.368359,Chinese Restaurant


In [89]:
#Cluster 2 area

mergedData.loc[(mergedData['Cluster Labels'] ==2) & (mergedData['Venue Category'] == 'Chinese Restaurant') ]

,Neighborhoods,Chinese Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
38,"University of Toronto, Harbord",0.028571,2,43.662696,-79.400049,River Tai Restaurant,43.662902,-79.403167,Chinese Restaurant


In [90]:
#Cluster 3 area

mergedData.loc[(mergedData['Cluster Labels'] ==3) & (mergedData['Venue Category'] == 'Chinese Restaurant') ]

,Neighborhoods,Chinese Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category


In [92]:
#Cluster 0 area

mergedData.loc[(mergedData['Cluster Labels'] ==0) & (mergedData['Venue Category'] == 'Chinese Restaurant') ]

,Neighborhoods,Chinese Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
37,"Toronto Dominion Centre, Design Exchange",0.01,0,43.647177,-79.381576,Szechuan Express,43.646973,-79.379549,Chinese Restaurant
13,"Garden District, Ryerson",0.01,0,43.657162,-79.378937,GB Hand-Pulled Noodles,43.656434,-79.383783,Chinese Restaurant
14,"Harbourfront East, Union Station, Toronto Islands",0.01,0,43.640816,-79.381752,Pearl Harbourfront,43.638157,-79.380688,Chinese Restaurant


### Ideal Chinese restaurant location are near Design Exchange, Toronto Dominion Centre, Garden District,  Union Station, Toronto Islands.
Looking at nearby venues it seems cluster 0 might be a good location as there are not a lot of Chinese restaurants in these areas. Therefore, this project recommends the entrepreneur to open an authentic Chinese restaurant in these locations.